In [282]:
import numpy as np
import copy
import random
from itertools import product
from tictactoe import TicTacToe
from time import time

# Zadanie 3
## Damian Baraniak 324851
#### WSI-24L-G104
Celem zadania jest implementacja algorytmów minimax oraz poprawionej wersji z odcięciem $\alpha - \beta$ do gry kółko i krzyżyk.
Przyjęto wersje, w której użytkownik gra z programem.
Aby zagrać należy wykonać plik `tictactoe.py`, zmiana poziomu trudności(głębokości przeszukiwania) znajduje się w wywołaniu konstruktora.

Dla ułatwienia implementacji użytkownik gra zawsze jako O, natomiast komputer zawsze jako X, kolejność jest losowa.

## Mini-Max w wersji podstawowej
### Opis działania algorytmu
Do działania, algorytm potrzebuje dostarczonego aktualnego stanu gry, głębokości na jakiej ma przeszukiwać przestrzeń rozwiązań oraz informację czyj ruch jest teraz wykonywany.
#### Stany końcowe gry
Pierwszą wykonywaną operacją jest sprawdzenie, czy podany stan gry nie oznacza zakończenia gry, albo dotarcia już do maksymalnej głębokości przeszukiwania. Jeśli jeden z tych warunków jest prawdziwy, ustawienie planszy podlega ocenie przez heurystyczną funkcję wypłaty. W przypadku implementacji zdecydowano się na zawarcie wszystkich czynności w jednej funkcji. Funkcja wypłaty analizuje trzy możliwe sytuacje:
- Jeśli gra nie uległa jeszcze zakończeniu zostaje zwrócone 0
- Jeśli gra zakończyła się remisem, jest zwracana bardzo mała wartość 0.0001 o znaku zależnym od tego kto się rusza.
- Jeśli aktualny stan kończy grę zwycięstwem zawodnika zostaje zwrócona wartość 10  o znaku zależnym od tego kto się rusza.
Aby zachęcić algorytm do wybierania zakończeń bliższych, wynik zwrócony przez funkcję jest mnożony razy głębokość jaka jeszcze pozostała do sprawdzenia.

In [283]:
def check_board(board:np.ndarray,who_is_moving:int):
    result = who_is_moving*10
    if np.count_nonzero(board)==9:
        return 0.0001*who_is_moving
    for row in board:
        if (row==who_is_moving).sum()==3:
            return result
    for row in board.transpose():
        if (row==who_is_moving).sum()==3:
            return result
    if (board.diagonal()==who_is_moving).sum()==3:
        return result
    elif (np.fliplr(board).diagonal()==who_is_moving).sum()==3:
        return result
    return 0

#### Generowanie nowych ruchów
Jeśli algorytm nie znalazł się w stanie końcowym generowana jest lista wszystkich dostępnych kolejnych ruchów dla kolejnego zawodnika.

In [284]:
def generate_next_moves(board:np.ndarray):
    states = []
    comb =  product([0,1,2],[0,1,2])
    for i in comb:
        if not board[*i]:
            states.append(i)
    return states


Następnie iterując przez wszystkie możliwe ruchy jest tworzona tymczasowa wersja plansza zaktualizowana o aktualny ruch. Dla tak przygotowanej planszy jest wywoływany rekurencyjnie algorytm minimax, z nową głębokością oraz ruszającym się zawodnikiem. W końcu taki alogrytm dojdzie do końca i zacznie zwracać wartości do góry. Z racji, że zakładamy grę obydwu graczy za optymalną to taką wartość ruchu wybieramy. Tak przygotowany algorytm zwraca ocenę wybranego ruchu.

### Implementacja algorytmu w języku Python

In [285]:
def MiniMaxTicTacToe(board: np.ndarray, depth: int, move_max: bool):
    pay = check_board(board, (1 if not move_max else -1))
    if pay or not depth:
        return pay * (depth + 1)
    best = -1000 if move_max else 1000
    moves = generate_next_moves(board)
    for move in moves:
        new_board = copy.copy(board)
        new_board[*move] = 1 if move_max else -1
        if move_max:
            best = max(best, MiniMaxTicTacToe(new_board, depth - 1, not move_max))
        else:
            best = min(best, MiniMaxTicTacToe(new_board, depth - 1, not move_max))
    return best

## Alfa-Beta
Rekurencyjne przeszukiwanie przestrzeni może być czasochłonne, ale zakładając optymalną grę zawodników, można nie analizować niektórych gałęzi ponieważ istnieje pewność, że nigdy nie zostaną wybrane.
### Różnice z wersją podstawową
Algorytmy nie różnią się między sobą do wygenerowania następników stanu planszy. Dalej, wykorzystywane są zmienne pomocnicze $\alpha$ i $\beta$, opisujące aktualnie najlepszą możliwość odpowiednio dla gracza Max i Min. Porównując wartości węzłów z aktualnie najlepszymi można z pewnością przyciąć drzewo, wiedząc, że konkretne ruchy nie mają podstaw zostać wybrane.
### Implementacja w języku Python:

In [286]:
def AlfaBeta(board: np.ndarray, depth: int, move_max: bool, alfa=-1e6, beta=1e6):
    pay = check_board(board, (1 if not move_max else -1))
    if pay or not depth:
        return pay * (depth + 1)
    moves = generate_next_moves(board)
    if move_max:
        for move in moves:
            new_board = copy.copy(board)
            new_board[*move] = 1 if move_max else -1
            alfa = max(alfa, AlfaBeta(new_board, depth - 1, not move_max, alfa, beta))
            if alfa >= beta:
                return alfa
        return alfa
    else:
        for move in moves:
            new_board = copy.copy(board)
            new_board[*move] = 1 if move_max else -1
            beta = min(beta, AlfaBeta(new_board, depth - 1, not move_max, alfa, beta))
            if alfa >= beta:
                return beta
        return beta

## Testowanie

### Przykładowe podejmowanie decyzji przez algorytm

In [287]:
# Głębokość przeszukania 4
game = TicTacToe(4)
temp_board1 = np.array([[1,0,-1],[-1,0,0],[1,0,0]])
game.board = copy.copy(temp_board1)
print(game)

X| |O
-----
O| | 
-----
X| | 


In [288]:
game.who_is_moving = game.COMPUTER
game.computer_move(algorithm=MiniMaxTicTacToe,debug= True)
print(game)
# Każdy możliwy ruch z odpowiadającą mu oceną, należy pamiętać, że indeksowanie zaczyna się od 0

{(1, 1): 0.0001, (2, 2): 30, (2, 1): 0.0001, (1, 2): 0.0001, (0, 1): -20}
X| |O
-----
O| | 
-----
X| |X


In [289]:
temp_board2 = np.array([[-1,0,-1],[0,0,0],[0,0,1]])
game.board = copy.copy(temp_board2)
print(game)

O| |O
-----
 | | 
-----
 | |X


In [290]:
game.who_is_moving = game.COMPUTER
game.computer_move(algorithm=MiniMaxTicTacToe,debug= True)
print(game)
# Każdy możliwy ruch z odpowiadającą mu oceną

{(2, 1): -40, (0, 1): -20, (1, 1): -40, (1, 0): -40, (1, 2): -40, (2, 0): -40}
O|X|O
-----
 | | 
-----
 | |X


#### Sekwencja ruchów

In [291]:
temp_board3 = np.array([[1,0,-1],[0,-1,0],[0,0,1]])
game.board = copy.copy(temp_board3)
print(game)

X| |O
-----
 |O| 
-----
 | |X


In [292]:
game.who_is_moving = game.COMPUTER
game.computer_move(algorithm=MiniMaxTicTacToe,debug= True)
print(game)

{(0, 1): -40, (2, 1): -40, (1, 0): -40, (2, 0): 30, (1, 2): -40}
X| |O
-----
 |O| 
-----
X| |X


In [293]:
# Przykładowy ruch gracza
game.board[2,1] = game.PLAYER
print(game)

X| |O
-----
 |O| 
-----
X|O|X


In [294]:
# Algorytm kończący grę
game.who_is_moving = game.COMPUTER
game.computer_move(algorithm=MiniMaxTicTacToe,debug= True)
print(game)

{(0, 1): 0.00030000000000000003, (1, 2): -40, (1, 0): 50}
X| |O
-----
X|O| 
-----
X|O|X


### Czas przeszukiwania a zadana głębokość i wersja algorytmu
Oczywiście w późniejszych stadiach gry, często nie ma dość możliwości aby sprawdzać dość głęboko.

#### Prosty MiniMax

In [295]:
# Dla każdej gry tablica początkowa jest taka sama.
temp_board_t = np.array([[0,0,0],[0,-1,0],[0,0,0]])

# Głębokość 2
game2 = TicTacToe(2)
game2.board = copy.copy(temp_board_t)
game2.who_is_moving = TicTacToe.COMPUTER

# Głębokość 4
game4 = TicTacToe(4)
game4.board = copy.copy(temp_board_t)
game4.who_is_moving = TicTacToe.COMPUTER

# Głębokość 8
game8 = TicTacToe(8)
game8.board = copy.copy(temp_board_t)
game8.who_is_moving = TicTacToe.COMPUTER


print("Stan początkowy")
print(game2)
print("\n")

ts2 = time()
game2.computer_move(MiniMaxTicTacToe,debug=True)
te2 = time()
print(f"Dla głębokości 2 po czasie {te2-ts2:.5f}s:")
print(game2)
print("\n")
ts4 = time()
game4.computer_move(MiniMaxTicTacToe,debug=True)
te4 = time()
print(f"Dla głębokości 4 po czasie {te4-ts4:.5f}s:")
print(game4)
print("\n")

ts8 = time()
game8.computer_move(MiniMaxTicTacToe,debug=True)
te8 = time()
print(f"Dla głębokości 8 po czasie {te8-ts8:.5f}s:")
print(game8)

Stan początkowy
 | | 
-----
 |O| 
-----
 | | 


{(0, 0): 0, (2, 1): 0, (0, 2): 0, (1, 0): 0, (2, 0): 0, (2, 2): 0, (1, 2): 0, (0, 1): 0}
Dla głębokości 2 po czasie 0.01505s:
X| | 
-----
 |O| 
-----
 | | 


{(2, 1): 0, (2, 2): 0, (1, 2): 0, (1, 0): 0, (0, 1): 0, (2, 0): 0, (0, 2): 0, (0, 0): 0}
Dla głębokości 4 po czasie 0.25117s:
 | | 
-----
 |O| 
-----
 |X| 




{(1, 2): -40, (0, 0): -0.0002, (2, 2): -0.0002, (1, 0): -40, (2, 1): -40, (0, 2): -0.0002, (2, 0): -0.0002, (0, 1): -40}
Dla głębokości 8 po czasie 1.25580s:
X| | 
-----
 |O| 
-----
 | | 


In [296]:
# Dla każdej gry tablica początkowa jest taka sama.
temp_board_t = np.array([[1,0,0],[0,0,0],[0,0,-1]])

# Głębokość 2
game2 = TicTacToe(2)
game2.board = copy.copy(temp_board_t)
game2.who_is_moving = TicTacToe.COMPUTER

# Głębokość 4
game4 = TicTacToe(4)
game4.board = copy.copy(temp_board_t)
game4.who_is_moving = TicTacToe.COMPUTER

# Głębokość 8
game8 = TicTacToe(8)
game8.board = copy.copy(temp_board_t)
game8.who_is_moving = TicTacToe.COMPUTER


print("Stan początkowy")
print(game2)
print("\n")

ts2 = time()
game2.computer_move(MiniMaxTicTacToe,debug=True)
te2 = time()
print(f"Dla głębokości 2 po czasie {te2-ts2:.5f}s:")
print(game2)
print("\n")
ts4 = time()
game4.computer_move(MiniMaxTicTacToe,debug=True)
te4 = time()
print(f"Dla głębokości 4 po czasie {te4-ts4:.5f}s:")
print(game4)
print("\n")

ts8 = time()
game8.computer_move(MiniMaxTicTacToe,debug=True)
te8 = time()
print(f"Dla głębokości 8 po czasie {te8-ts8:.5f}s:")
print(game8)

Stan początkowy
X| | 
-----
 | | 
-----
 | |O


{(1, 0): 0, (1, 2): 0, (2, 0): 0, (2, 1): 0, (1, 1): 0, (0, 1): 0, (0, 2): 0}
Dla głębokości 2 po czasie 0.00716s:
X| | 
-----
X| | 
-----
 | |O


{(1, 2): 0, (1, 0): 0, (2, 0): 10, (0, 2): 10, (2, 1): 0, (1, 1): 0, (0, 1): 0}
Dla głębokości 4 po czasie 0.09253s:
X| | 
-----
 | | 
-----
X| |O


{(1, 1): 0.00030000000000000003, (1, 0): -40, (2, 1): 0.00030000000000000003, (0, 2): 50, (2, 0): 50, (0, 1): -40, (1, 2): 0.00030000000000000003}
Dla głębokości 8 po czasie 0.17932s:
X| |X
-----
 | | 
-----
 | |O


#### Odcięcie alfa beta

In [297]:
# Dla każdej gry tablica początkowa jest taka sama.
temp_board_t = np.array([[0,0,0],[0,-1,0],[0,0,0]])

# Głębokość 2
game2 = TicTacToe(2)
game2.board = copy.copy(temp_board_t)
game2.who_is_moving = TicTacToe.COMPUTER

# Głębokość 4
game4 = TicTacToe(4)
game4.board = copy.copy(temp_board_t)
game4.who_is_moving = TicTacToe.COMPUTER

# Głębokość 8
game8 = TicTacToe(8)
game8.board = copy.copy(temp_board_t)
game8.who_is_moving = TicTacToe.COMPUTER


print("Stan początkowy")
print(game2)
print("\n")

ts2 = time()
game2.computer_move(AlfaBeta,debug=True)
te2 = time()
print(f"Dla głębokości 2 po czasie {te2-ts2:.5f}s:")
print(game2)
print("\n")
ts4 = time()
game4.computer_move(AlfaBeta,debug=True)
te4 = time()
print(f"Dla głębokości 4 po czasie {te4-ts4:.5f}s:")
print(game4)
print("\n")

ts8 = time()
game8.computer_move(AlfaBeta,debug=True)
te8 = time()
print(f"Dla głębokości 8 po czasie {te8-ts8:.5f}s:")
print(game8)

Stan początkowy
 | | 
-----
 |O| 
-----
 | | 


{(1, 0): 0, (2, 0): 0, (1, 2): 0, (0, 1): 0, (0, 0): 0, (0, 2): 0, (2, 2): 0, (2, 1): 0}
Dla głębokości 2 po czasie 0.00652s:
 | | 
-----
X|O| 
-----
 | | 


{(2, 2): 0, (0, 1): 0, (2, 0): 0, (0, 2): 0, (0, 0): 0, (1, 2): 0, (1, 0): 0, (2, 1): 0}
Dla głębokości 4 po czasie 0.06605s:
 | | 
-----
 |O| 
-----
 | |X


{(1, 0): -40, (0, 0): -0.0002, (2, 1): -40, (0, 1): -40, (2, 2): -0.0002, (1, 2): -40, (2, 0): -0.0002, (0, 2): -0.0002}
Dla głębokości 8 po czasie 0.21509s:
X| | 
-----
 |O| 
-----
 | | 


In [298]:
# Dla każdej gry tablica początkowa jest taka sama.
temp_board_t = np.array([[1,0,0],[0,0,0],[0,0,-1]])

# Głębokość 2
game2 = TicTacToe(2)
game2.board = copy.copy(temp_board_t)
game2.who_is_moving = TicTacToe.COMPUTER

# Głębokość 4
game4 = TicTacToe(4)
game4.board = copy.copy(temp_board_t)
game4.who_is_moving = TicTacToe.COMPUTER

# Głębokość 8
game8 = TicTacToe(8)
game8.board = copy.copy(temp_board_t)
game8.who_is_moving = TicTacToe.COMPUTER


print("Stan początkowy")
print(game2)
print("\n")

ts2 = time()
game2.computer_move(AlfaBeta,debug=True)
te2 = time()
print(f"Dla głębokości 2 po czasie {te2-ts2:.5f}s:")
print(game2)
print("\n")
ts4 = time()
game4.computer_move(AlfaBeta,debug=True)
te4 = time()
print(f"Dla głębokości 4 po czasie {te4-ts4:.5f}s:")
print(game4)
print("\n")

ts8 = time()
game8.computer_move(AlfaBeta,debug=True)
te8 = time()
print(f"Dla głębokości 8 po czasie {te8-ts8:.5f}s:")
print(game8)

Stan początkowy
X| | 
-----
 | | 
-----
 | |O


{(1, 1): 0, (1, 0): 0, (2, 1): 0, (2, 0): 0, (0, 2): 0, (1, 2): 0, (0, 1): 0}
Dla głębokości 2 po czasie 0.00450s:
X| | 
-----
 |X| 
-----
 | |O


{(2, 1): 0, (1, 2): 0, (1, 0): 0, (1, 1): 0, (0, 1): 0, (2, 0): 10, (0, 2): 10}
Dla głębokości 4 po czasie 0.02807s:
X| | 
-----
 | | 
-----
X| |O


{(2, 0): 50, (0, 1): -40, (2, 1): 0.00030000000000000003, (1, 1): 0.00030000000000000003, (1, 2): 0.00030000000000000003, (1, 0): -40, (0, 2): 50}
Dla głębokości 8 po czasie 0.05040s:
X| | 
-----
 | | 
-----
X| |O


## Podsumowanie i wnioski
Heurystyczna funkcja wypłaty pozostawia wiele do życzenia, dlatego działania algorytmu o małej głębokości i pustej tablicy wydaję się losowe. Dopiero przy większej głębkości jak 4 czy 8 dla algorytmu daje się zaobserwować skuteczne przeszukiwanie przestrzeni za rozwiązaniami.

Nie jest żadnym zaskoczeniem, że zwiększenie głębokości przeszukiwania wydłuża czas obliczeń, ponieważ jest sprawdzanie znacznie więcej węzłów. Przy późniejszych stadiach rozgrywki, z racji że gra w kółko i krzyżyk jest stosunkowo krótka, nie ma możliwości wyczerpania całej głębokości więc algorytm znacznie przyspiesza przy kilku polach do wypełnienia i jakość jego odpowiedzi, dla małej głębokości rośnie.

Metoda Alfa Beta w znacznym stopniu skraca działanie algorytmu. Przycinając drzewo przeszukań o gałęzie, które na pewno nie zostaną wybrane udało się w znacznym stopniu zmniejszyć czas obliczeń.